<a href="https://colab.research.google.com/github/elliwadler/image_classification/blob/main/cat_dog_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import urllib.request
import zipfile

# Download the dataset zip file
url = "https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip"
filename = "cats_and_dogs_filtered.zip"
urllib.request.urlretrieve(url, filename)

# Extract the zip file
with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall()

url = "https://drive.google.com/file/d/1HZZ9Kw4s8dZA9jN_J0En3p0a41pW2kEX/view?usp=sharing"
filename = "cats_dogs_classifier.h5"
urllib.request.urlretrieve(url, filename)


('cats_dogs_classifier.h5', <http.client.HTTPMessage at 0x7f8b86215250>)

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [ ]:
train_dir = 'cats_and_dogs_filtered/train'
validation_dir = 'cats_and_dogs_filtered/validation'

# TensorFlow - Klasse: erstellen einer Pipeline für das Lesen von Bildern aus Verzeichnis, Bilder in Echtzeit währen des Trainings verändern (Drehung, Zuschneiden, Spiegeln, ...)
# Erleichtert das Training von Deep-Learning-Modellen mit Bildern, da sie eine effiziente Möglichkeit bietet, eine große Anzahl von Bildern automatisch zu verarbeiten und zu augmentieren.
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)


In [ ]:
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')


Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=tf.optimizers.RMSprop(learning_rate=1e-4),
              metrics=['accuracy'])

In [ ]:
history = model.fit(
      train_generator,
      steps_per_epoch=100,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=50)


Epoch 1/30
100/100 [==============================] - 137s 1s/step - loss: 0.6894 - accuracy: 0.5410 - val_loss: 0.6844 - val_accuracy: 0.5490
Epoch 2/30
100/100 [==============================] - 136s 1s/step - loss: 0.6731 - accuracy: 0.5910 - val_loss: 0.6570 - val_accuracy: 0.5980
Epoch 3/30
100/100 [==============================] - 141s 1s/step - loss: 0.6520 - accuracy: 0.6200 - val_loss: 0.6875 - val_accuracy: 0.5700
Epoch 4/30
100/100 [==============================] - 136s 1s/step - loss: 0.6276 - accuracy: 0.6445 - val_loss: 0.6149 - val_accuracy: 0.6650
Epoch 5/30
100/100 [==============================] - 138s 1s/step - loss: 0.5917 - accuracy: 0.6970 - val_loss: 0.5841 - val_accuracy: 0.6970
Epoch 6/30
100/100 [==============================] - 133s 1s/step - loss: 0.5547 - accuracy: 0.7135 - val_loss: 0.5857 - val_accuracy: 0.6950
Epoch 7/30
100/100 [==============================] - 139s 1s/step - loss: 0.5393 - accuracy: 0.7195 - val_loss: 0.5630 - val_accuracy: 0.7160

In [ ]:
import pandas as pd

history_df = pd.DataFrame(model.history.history)
history_df.to_csv('train_history.csv', index=False)
model.save('cats_dogs_classifier.h5')

In [10]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

# Load the saved model
model = tf.keras.models.load_model('drive/MyDrive/cats_dogs_classifier.h5')

# Load the image and resize to the target size
img_path = '/content/cats_and_dogs_filtered/validation/dogs/dog.2004.jpg'
img = image.load_img(img_path, target_size=(150, 150))

# Convert the image to a numpy array and normalize
img_array = image.img_to_array(img)
img_array /= 255.

# Expand the dimensions of the image array to match the model input shape
img_array = np.expand_dims(img_array, axis=0)

# Predict the class of the image
pred = model.predict(img_array)
pred_probability = pred[0][0]

if pred[0][0] > 0.5:
    print("The image is a dog with a probability of {:.2f}%".format(pred_probability * 100))
else:
    print("The image is a cat with a probability of {:.2f}%".format(pred_probability * 100))

1/1 [==============================] - 0s 120ms/step
The image is a dog with a probability of 99.96%
